![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=Health/CALM/CALM-moving-out-2.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# CALM - Moving Out Project
## Part 2 - Income

📗Now let's take a look at your potential income. First, some definitions:

### Paycheque Definitions
#### Gross Income (pay/earnings)
The amount of income/earnings, for any pay period, before deductions
 
#### Net income (pay/earnings)
The amount of income/earnings, for any pay period, after deductions (Take home pay)

#### CPP - Canada Pension Plan
Pensionable earnings received later in life (standard age is 65). Percentage of payment to CPP changes frequently but has ranged from 2-6%. 

#### Income Tax
A deduction paid to the Federal and Provincial government for taxes
 
#### LTD
A deduction for Long Term Disability insurance
 
#### Union Dues
Fees paid for membership in a union
 
#### Bonds
An investment in which a business or government pays a set interest rate
 
#### Advance Earnings
Deducted money that was received in advance of the pay cheque
 
#### Overtime Earnings
Pay received for working over 8 hours a day or 44 hours a week, whichever is greater

📗 Run the cell below to import libraries required for the notebook. 

In [ ]:
%store -r
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from functions import *
import plotly.express as px
import numpy as np
print("Libraries imported.")

### Calculating Net Income

📗 Click on the code cell below, then click the `Run` button on the toolbar to calculate your net income. You may also change some values in cells that contain (✏️) to see how the results change.

In [ ]:
dataset = pd.read_csv("https://raw.githubusercontent.com/callysto/data-files/main/Health/CALM/Moving-out-2/income.csv") #(csv downloaded from https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=9810045201)
dataset

📗 The dataset will be "cleaned" for ease of visualization and data-analysis later on. Continue to run the next few cells. 

In [ ]:
for column in dataset.columns[1:]:
    if dataset[column].dtype == 'object':
        dataset[column] = dataset[column].str.replace(',', '', regex=True)
        dataset[column] = pd.to_numeric(dataset[column], errors='coerce').astype('Int64')
        
pattern = r'^\d+\s+'
dataset['Occupation'] = dataset['Occupation'].str.replace(pattern, '', regex=True)
dataset

Below is a list of occupations found in the dataset which are used in the next 2 ✏️ cells.

In [ ]:
list_of_occupations = dataset['Occupation'].unique()
for occupation in list_of_occupations:
    print(occupation)

✏️ Input a job you'd like to research more about by changing the value of the variable **search_job** below. 

In [16]:
# ✏️ Change search_job to any job you would like to research more about in the dataset above
search_job = 'computer'

filtered_df = dataset[dataset['Occupation'].str.contains(search_job, case=False)][['Occupation', 'Average employment income']]

jobs_fig = px.scatter(
    filtered_df,
    x='Occupation',
    y='Average employment income',
    color='Occupation',
    title="Average employment salary by occupation (hover for occupation)"

)
jobs_fig.update_xaxes(showticklabels=False)
jobs_fig.update_layout(xaxis_title=None, showlegend=False)

jobs_fig.update_traces(marker=dict(size=12,line=dict(width=2,color='DarkSlateGrey')),selector=dict(mode='markers'))
jobs_fig.add_annotation(text='Occupation',xref='paper',yref='paper',x=0.5,y=-0.1,showarrow=False,font=dict(size=14))
jobs_fig.show()

✏️ Visualizing the average salaries of all job-types across Canada helps to give a better perspective of wages in Canada.

In [ ]:
# Comparing 2 jobs salaries by occupation 
# ✏️ Edit occupation_1 and occupation_2 below to occupations that you'd like to compare
occupation_1 = 'physician'
occupation_2 = 'lawyer'

filtered_df = dataset[dataset['Occupation'].str.contains(occupation_1, case=False)][['Occupation', 'Average employment income']]
filtered_df2 = dataset[dataset['Occupation'].str.contains(occupation_2, case=False)][['Occupation', 'Average employment income']]

merged_df = pd.concat([filtered_df, filtered_df2])

occupation_fig = px.scatter(
    merged_df,
    x="Occupation",
    y="Average employment income",
    color="Occupation",
).show()

📗 Select a salary range that you'd like to achieve. Based on the range, you will be presented 15 different potential job opportunities that may interest you. 

The slider (✏️) can be adjusted in the widget output below.

In [ ]:
salary_range = create_widget('IntRangeSlider','Range of Salary that I want to make:',[0,100000],0,250000,2500)
box = widgets.VBox([salary_range])

display(box)

In [ ]:
min_salary, max_salary = salary_range.value

filtered_dataset = dataset[
    (dataset['Average employment income'] >= min_salary) &
    (dataset['Average employment income'] <= max_salary)
]

# List the top 15 rows of the filtered DataFrame
top_15 = filtered_dataset.sample(15)[['Occupation', "Average employment income"]]

# Display the top 15 rows
display(top_15.reset_index(drop=True))
salary_btn = widgets.Button(description='Click when Finished')

✏️ Edit the values of **wagePerHour**, **hoursPerDay**, and **workdaysPerMonth** to calculate your gross income (total earnings before taxes and/or other deductions), government deductions, and net income (earnings after/or other deductions)

In [ ]:
# Create a widget for this portion

# ✏️ Edit the values below to accurately reflect your wage, hours worked, and days worked per month
wagePerHour = 15  # this is your wage in $/hour
hoursPerDay = 8
workdaysPerMonth = 21

grossIncome = wagePerHour * hoursPerDay * workdaysPerMonth
print('Your gross income is $', grossIncome, 'per month.')

incomeTax = .15 + .10   # assume federal income tax is 15% and provincial is 10%
cpp = .0595             # assume Canada Pension Plan is 5.95%
ei = .0163             # assume Employment Insurance is 1.63%
unionDues = .0075       # 0.75% sounds reasonable for union dues

deductions = grossIncome * (incomeTax + cpp + ei + unionDues)
print('$', '{:.2f}'.format(deductions), ' will be taken off your paycheck.')

netIncome = grossIncome - deductions
print('Your net income is $', '{:.2f}'.format(netIncome), 'per month.')

# store that value in memory for later notebooks
%store netIncome

## Graphing Income
📗
We can also look at how your net income (take-home pay) changes based on your hourly wage. We will use [2023 taxation rates](https://www.canada.ca/en/revenue-agency/services/tax/individuals/frequently-asked-questions-individuals/canadian-income-tax-rates-individuals-current-previous-years.html) as well as [EI](https://www.canada.ca/en/revenue-agency/services/tax/businesses/topics/payroll/payroll-deductions-contributions/employment-insurance-ei/ei-premium-rates-maximums.html) and [CPP](https://www.canada.ca/en/revenue-agency/services/tax/businesses/topics/payroll/payroll-deductions-contributions/canada-pension-plan-cpp/cpp-contribution-rates-maximums-exemptions.html) rates. `Run` the cell below (without editing it) to generate a graph. 
</div>

In [ ]:
wages = []
grossIncomes = []
netIncomes = []

for wage in range(15, 250):
    wages.append(wage)
    grossAnnualIncome = wage * 8 * 240
    grossIncomes.append(grossAnnualIncome)
    incomeTax = calculateFederalTax(grossAnnualIncome) + calculateProvincialTax(grossAnnualIncome)
    eiPremium = calculateEI(grossAnnualIncome)
    cppPremium = calculateCPP(grossAnnualIncome)
    netIncome = grossAnnualIncome - (incomeTax + eiPremium + cppPremium)
    netIncomes.append(netIncome)

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=wages, y=grossIncomes, name='Gross Income'))
fig.add_trace(go.Scatter(x=wages, y=netIncomes, name='Net Income'))
fig.update_layout(
    title=go.layout.Title(text='Net Income vs Hourly Wage'),
    xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Hourly Wage')),
    yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Income')))
fig.show()

📗 The graph shows that the difference between gross income and net income (after deductions) increases as wage increases. For more information about this, you may want to read about [progressive taxation](https://en.wikipedia.org/wiki/Progressive_tax). Feel free to scroll/zoom

You have now completed this section. Proceed to [section 3](./CALM-moving-out-3.ipynb)

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)